In [ ]:
import tweepy
from twarc import Twarc
from kafka import KafkaProducer
import json


In [ ]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

In [ ]:
# currently, having two topics: twitter & twittertry 
producer = KafkaProducer(bootstrap_servers='localhost:9092')
# topic_name = "twittertry"
topic_name = "twitter"

In [ ]:
# create a new csv file that only contains the tweet Id
import pandas as pd
df = pd.read_csv('corona_tweets/corona_tweets_470gcp.csv', delimiter=",")
tweetId = df[df.columns[0]]
tweetId.to_csv('tweetId.csv', index=False)


In [ ]:
# how many tweets 
import csv
file = open("tweetId.csv")
reader = csv.reader(file)
lines= len(list(reader))


In [ ]:
# using kafka 
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)
count = 0
state_count = 0
# hydrate is a generator function 
for tweet in t.hydrate(open('tweetId.csv')):

    t = {'text': tweet['full_text'], 
     'user_location': tweet['user']['location']
     }
#     t = {'text': tweet['full_text'], 
#          'user_location': tweet['user']['location'], 
#          'tweet_place': tweet['place']}
# geo location test 
#     test = {
#         'user_location': tweet['user']['location'], 
#          'tweet_place': tweet['place'],
#         'coordinates': tweet['coordinates']
#     }
#     if loc_filter(t['user_location']) != None:
#         print(loc_filter(t['user_location']))
#         print(t['text'])
#         state_count += 1
#     if test['coordinates'] != None:
#         state_count += 1
#         print(test['coordinates'])
    count +=1 
    print(count)
# Producer: produce json strings and send out 
    producer.send(topic_name, json.dumps(t).encode('utf-8'))
    if count >= 100:
#         print(state_count)
        break

In [ ]:
# without using kafka

from csv import DictWriter
headersCSV = ['text','location']  

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)
count = 0
state_count = 0
# hydrate is a generator function 
with open('geo_tweet_469.csv', 'w', newline='') as f_object:

    dictwriter_object = DictWriter(f_object, fieldnames=headersCSV)

    for tweet in t.hydrate(open('tweetId.csv')):

        count +=1 
        print(count)
        if loc_filter(tweet['user']['location']) != None:
            t = {
                'text': tweet['full_text'],
                'location': loc_filter(tweet['user']['location'])
            }
            print(t)

            dictwriter_object.writerow(t)
        if count >= 100000:
            break

In [ ]:
# read us cities csv
import pandas as pd
df = pd.read_csv('uscities.csv', delimiter=",")
df.head(5)


In [ ]:
# df["city"].head(5)
df.loc[df['city'] == 'Melbourne']
# find the corresponding row 
df.loc[df['state_id'] == 'CA']
# find if a value is in a column 
if(df['state_id'].str.contains('CA').any()):
    print("true")




In [ ]:
df['state_name'].apply(lambda state : state.upper()=='NEW YORK').any()

In [ ]:
# User location filter 
# By testing with big enough samples:
# Aronud 8% of tweets have infomation of the user's state
# Less than 0.5% of the tweets contain information like tweet's coordinates or user's place
# Therefore, for now, we focus on the states information of the tweets 

def loc_filter(loc):
    if ',' in loc:
        s1 = loc.split(',')[0].strip().upper()
        s2 = loc.split(',')[1].strip().upper()
#         check if part before ',' is CA for instance 
        if(df['state_id'].apply(lambda state_id : state_id==s1).any()):
            return(s1)
#         check if part after ',' is CA for instance 
        elif(df['state_id'].apply(lambda state_id : state_id==s2).any()):
            return(s2)
#         check if part before ',' is CALIFORNIA for instance 
        elif(df['state_name'].apply(lambda state_name : state_name.upper()==s1).any()):
            return(s1)
#         check if part after ',' is CALIFORNIA for instance 
        elif(df['state_name'].apply(lambda state_name : state_name.upper()==s2).any()):
            return(s2)
        
loc_filter('xxx, VA')

In [ ]:
# t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)
# count = 0
# # hydrate is a generator function 
# for tweetId in open('corona_tweets_441.csv'):
# #     print(tweet['full_text'])
# #     print("User location: ", tweet['user']['location'])
# #     print("Tweet place:", tweet['place'])
#     tweetId = tweetId.rpartition(',')[0]
#     tweet = t.hydrate(tweetId)
#     print(list(tweet))
# #     info = {'text': tweet['full_text'], 
# #          'user_location': tweet['user']['location'], 
# #          'tweet_place': tweet['place']}
#     print(count)
#     count +=1 
# #     print(tweetId.rpartition(',')[0])
# # Producer: produce json strings and send out 
# #     producer.send(topic_name, json.dumps(t).encode('utf-8'))
#     if count >= 500:
#         break